<a href="https://colab.research.google.com/github/lsmanoel/pythonLearningAndDeepLearning/blob/master/Tensorflow/Tensorflow_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A simple Net using Tensorflow to recognize handwritten digits - Deep Learning

A simple exemple by [digitalocean.com](https://www.digitalocean.com/community/tutorials/how-to-build-a-neural-network-to-recognize-handwritten-digits-with-tensorflow)
to implement a Deep Learning base Neural Network. Thks **Ellie Birbeck** for this didactic demostration. The Net that will be implemented has the architectures shown in the image: 

![Source: digitalocean.com](https://assets.digitalocean.com/articles/handwriting_tensorflow_python3/cnwitLM.png)
[Source](https://www.digitalocean.com/community/tutorials/como-construir-uma-rede-neural-para-reconhecer-digitos-manuscritos-com-o-tensorflow-pt) 


In [8]:
import tensorflow as tf

#-------------------------------------------------------------------------------
# Load Dataset:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True) # y labels are oh-encoded

n_train = mnist.train.num_examples # 55,000
n_validation = mnist.validation.num_examples # 5000
n_test = mnist.test.num_examples # 10,000

#-------------------------------------------------------------------------------
# Net Config:
n_input = 784   # input layer (28x28 pixels)
n_hidden1 = 512 # 1st hidden layer
n_hidden2 = 256 # 2nd hidden layer
n_hidden3 = 128 # 3rd hidden layer
n_output = 10   # output layer (0-9 digits)

learning_rate = 1e-4
n_iterations = 1000
batch_size = 128
dropout = 0.5


#-------------------------------------------------------------------------------
# Net Architectures - The Graph:
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_output])
keep_prob = tf.placeholder(tf.float32)

weights = {
    'w1': tf.Variable(tf.truncated_normal([n_input, n_hidden1], stddev=0.1)),
    'w2': tf.Variable(tf.truncated_normal([n_hidden1, n_hidden2], stddev=0.1)),
    'w3': tf.Variable(tf.truncated_normal([n_hidden2, n_hidden3], stddev=0.1)),
    'out': tf.Variable(tf.truncated_normal([n_hidden3, n_output], stddev=0.1)),
}


biases = {
    'b1': tf.Variable(tf.constant(0.1, shape=[n_hidden1])),
    'b2': tf.Variable(tf.constant(0.1, shape=[n_hidden2])),
    'b3': tf.Variable(tf.constant(0.1, shape=[n_hidden3])),
    'out': tf.Variable(tf.constant(0.1, shape=[n_output]))
}

layer_1 = tf.add(tf.matmul(X, weights['w1']), biases['b1'])
layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
layer_3 = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
layer_drop = tf.nn.dropout(layer_3, keep_prob)
output_layer = tf.matmul(layer_3, weights['out']) + biases['out']

#-------------------------------------------------------------------------------
# Net Fit Functions:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output_layer))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

#-------------------------------------------------------------------------------
# Net Evaluate Functions:
correct_pred = tf.equal(tf.argmax(output_layer, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

#-------------------------------------------------------------------------------
# Run Config:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

#-------------------------------------------------------------------------------
# Run train on mini batches
for i in range(n_iterations):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    sess.run(train_step, feed_dict={X: batch_x, Y: batch_y, keep_prob:dropout})

    # print loss and accuracy (per minibatch)
    if i%100==0:
        minibatch_loss, minibatch_accuracy = sess.run([cross_entropy, accuracy], feed_dict={X: batch_x, Y: batch_y, keep_prob:1.0})
        print("Iteration", str(i), "\t| Loss =", str(minibatch_loss), "\t| Accuracy =", str(minibatch_accuracy))

#-------------------------------------------------------------------------------
# Run Test
test_accuracy = sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob:1.0})
print("\nAccuracy on test set:", test_accuracy)        

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Iteration 0 	| Loss = 3.3001502 	| Accuracy = 0.109375
Iteration 100 	| Loss = 0.5718136 	| Accuracy = 0.828125
Iteration 200 	| Loss = 0.44090348 	| Accuracy = 0.8828125
Iteration 300 	| Loss = 0.37062013 	| Accuracy = 0.8828125
Iteration 400 	| Loss = 0.3162076 	| Accuracy = 0.921875
Iteration 500 	| Loss = 0.23818952 	| Accuracy = 0.921875
Iteration 600 	| Loss = 0.35121247 	| Accuracy = 0.8671875
Iteration 700 	| Loss = 0.20537658 	| Accuracy = 0.9296875
Iteration 800 	| Loss = 0.38748917 	| Accuracy = 0.90625
Iteration 900 	| Loss = 0.24222265 	| Accuracy = 0.9453125

Accuracy on test set: 0.9149


In this exemple we successfully trained a neural network to classify the MNIST dataset with around 92% accuracy. Current state-of-the-art research achieves around 99% on this same problem, using more complex network architectures involving convolutional layers.

[Tensorflow Documentation](https://www.tensorflow.org/api_docs/python/tf)